In [20]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Ensure OSMnx returns buildings as GeoDataFrame
ox.settings.use_cache = True
ox.settings.log_console = True

# Query building footprints (works on all OSMnx 1.x+)
tags = {"building": True}

buildings = ox.features_from_place(
    "Teknaf Upazila, Cox's Bazar, Bangladesh",
    tags
)

# clean CRS + columns
buildings = buildings.to_crs("EPSG:4326")
buildings = buildings.reset_index(drop=True)[["geometry"]]

buildings.head()


,geometry
0,POINT (92.30048 20.83066)
1,POINT (92.3277 20.75943)
2,POINT (92.33084 20.75978)
3,POINT (92.33374 20.75898)
4,POINT (92.33394 20.76019)


In [21]:
# read CSV
shelters_df = pd.read_csv("shelter_final.csv")

# convert to GeoDataFrame
shelters = gpd.GeoDataFrame(
    shelters_df,
    geometry=gpd.points_from_xy(shelters_df.lon, shelters_df.lat),
    crs="EPSG:4326"
)

shelters.head()

,CS_ID,District,Upazila,union,lon,lat,Shelter Name (English),DRRO Capacity,geometry
0,cs_1,cox's bazar,teknaf,teknaf paurashava,92.290678,20.856927,baitus saraf muhammodia riadul jannah dakil ma...,1000.0,POINT (92.29068 20.85693)
1,cs_10,cox's bazar,teknaf,sabrang,92.332180,20.760671,sahaporirdip dakkin para center,1000.0,POINT (92.33218 20.76067)
2,cs_100,cox's bazar,ukhiya,palong khali,92.168040,21.189026,balokhali govt. primary school,400.0,POINT (92.16804 21.18903)
3,cs_101,cox's bazar,ukhiya,palong khali,92.168101,21.190423,balukhali kamenia high school,250.0,POINT (92.1681 21.19042)
4,cs_102,cox's bazar,ukhia,palong khali,92.169807,21.144842,farir beel alim madrasha,2500.0,POINT (92.16981 21.14484)


In [22]:
buildings_poly = buildings[buildings.geometry.type == "Polygon"].copy()


In [23]:
buildings_poly = buildings[buildings.geometry.geom_type.isin(["Polygon", "MultiPolygon"])].copy()


In [24]:
shelter_buffers = shelters.to_crs(3857).buffer(300).to_crs(4326)
shelter_buffers = gpd.GeoDataFrame(geometry=shelter_buffers, crs="EPSG:4326")


In [25]:
candidates = gpd.sjoin(
    buildings_poly,
    shelter_buffers,
    predicate="intersects",
    how="inner"
)


In [26]:
candidates["area_m2"] = candidates.to_crs(3857).area
candidates_large = candidates[candidates.area_m2 > 250]


In [27]:
candidate_shelters = candidates_large.reset_index(drop=True)
candidate_shelters



,geometry,index_right,area_m2
0,"POLYGON ((92.33216 20.76081, 92.33219 20.7608,...",1,414.438360
1,"POLYGON ((92.33188 20.7599, 92.33209 20.75989,...",1,291.341268
2,"POLYGON ((92.33267 20.7677, 92.33276 20.76771,...",119,453.628798
3,"POLYGON ((92.33267 20.7677, 92.33276 20.76771,...",117,453.628798
4,"POLYGON ((92.32559 20.77475, 92.32575 20.77476...",12,412.797771
...,...,...,...
738,"POLYGON ((92.24811 20.97281, 92.24811 20.97271...",132,299.032754
739,"POLYGON ((92.24799 20.97252, 92.24813 20.97252...",132,352.567877
740,"POLYGON ((92.24672 20.972, 92.24693 20.97199, ...",132,333.786949
741,"POLYGON ((92.24671 20.97223, 92.24693 20.97223...",132,628.910922


In [28]:
# Reproject to a metric CRS (Web Mercator)
candidates_proj = candidates.to_crs(3857)

# Compute centroids safely in meters
candidates_centroids = candidates_proj.centroid

# Add lon/lat back in WGS84
candidates["lon"] = candidates_centroids.to_crs(4326).x
candidates["lat"] = candidates_centroids.to_crs(4326).y
candidates[["lon", "lat", "area_m2", "index_right"]].to_csv(
    "candidate_shelters_points.csv", index=False
)

In [29]:
# Load the candidates file
df = pd.read_csv("candidate_shelters_points.csv")

# Step 1: Sort by area
df_sorted = df.sort_values("area_m2", ascending=False)

# Step 2: Group by shelter buffer ID and take the largest building for each
largest_per_shelter = df_sorted.groupby("index_right").head(1)

# Step 3: If there are more than 10 unique shelters, keep the top 10 largest buildings
top10 = (
    largest_per_shelter
    .sort_values("area_m2", ascending=False)
    .head(10)
    .reset_index(drop=True)
)

# Show result
top10


,lon,lat,area_m2,index_right
0,92.178256,21.030536,2778.967597,130
1,92.298485,20.870840,1351.647726,122
2,92.298485,20.870840,1351.647726,124
3,92.108871,21.115323,908.405275,151
4,92.253798,20.888386,857.546680,94
5,92.201904,20.974137,814.411237,150
6,92.299326,20.874741,774.814768,134
7,92.217517,21.079929,774.180078,86
8,92.297600,20.875853,753.536584,95
9,92.301621,20.864598,752.533996,82


In [30]:
# Original values
data = {
    "name": [
        "Baharchhara Union Parishad Complex",
        "IOM Teknaf Field Office",
        "INSAF hotel",
        "Parochial school",
        "Lombori",
        "Mathavanga Beat Office",
        "Teknaf Upazila Complex Adarsha School",
        "Kanjorpara Union Council Office",
        "Teknaf Upazila Health Complex",
        "Teknaf Municipality"
    ],
    "lon": [
        92.178256, 92.298485, 92.298485, 92.108871, 
        92.253798, 92.201904, 92.299326, 92.217517, 
        92.297600, 92.301621
    ],
    "lat": [
        21.030536, 20.870840, 20.870840, 21.115323,
        20.888386, 20.974137, 20.874741, 21.079929,
        20.875853, 20.864598
    ],
    "area_m2": [
        2778.967597, 1351.647726, 1351.647726, 908.405275,
        857.546680, 814.411237, 774.814768, 774.180078,
        753.536584, 752.533996
    ],
    "index_right": [130, 124, 122, 151, 94, 150, 134, 86, 95, 82]
}

df_named = pd.DataFrame(data)
df_named


,name,lon,lat,area_m2,index_right
0,Baharchhara Union Parishad Complex,92.178256,21.030536,2778.967597,130
1,IOM Teknaf Field Office,92.298485,20.870840,1351.647726,124
2,INSAF hotel,92.298485,20.870840,1351.647726,122
3,Parochial school,92.108871,21.115323,908.405275,151
4,Lombori,92.253798,20.888386,857.546680,94
5,Mathavanga Beat Office,92.201904,20.974137,814.411237,150
6,Teknaf Upazila Complex Adarsha School,92.299326,20.874741,774.814768,134
7,Kanjorpara Union Council Office,92.217517,21.079929,774.180078,86
8,Teknaf Upazila Health Complex,92.297600,20.875853,753.536584,95
9,Teknaf Municipality,92.301621,20.864598,752.533996,82
